In [1]:
import cv2 as cv
import copy
import numpy as np#添加模块和矩阵模块
import matplotlib.pyplot as plt
import os

In [13]:
def detectface(img,infile):
    infilename = os.path.split(infile)[1]
    #用"a"打开用于追加
    f = open("./val_output/ref.txt","a")
    #使用自己训练出的脸
    face_cascade = cv.CascadeClassifier('./detector/.xml')
    face_cascade.load('./detector/haarcascade_frontalface_default.xml')
    frame1=copy.copy(img)
    #人脸识别
    gray = cv.cvtColor(frame1, cv.COLOR_BGR2GRAY)
    # 脸
    faces = face_cascade.detectMultiScale(gray,1.1, 5) 
    for (x, y, w, h) in faces:
        cv.rectangle(frame1, (x, y), (x + w, y + h), (255, 0, 0), 2)
    #显示在图上
    #cv.imshow("facewindow",frame1)
    #cv.waitKey(0)
    
    #将数据写在输出的文档中
    f.write(infilename + " " + str(len(faces)) + " ")
    if len(faces) != 0:
        for (x, y, w, h) in faces:
            ##当有多个脸时，应该要排序！！！！！！！！！---注意！！！！
            f.write(str(x) + " " + str(y) + " " + str(w) + " " + str(h) + " ")
    f.write("\n")
    f.close()

In [3]:
# 一个窗口窗口显示多张图片：python3 + opencv3的版本。
# 传入的参数是：
# 1. 图片的集合（大小、通道数需要一致，否则黑屏）
# 2. 想显示到一张图片的大小
# 3. 图片间隔大小。

# 如果图片太多，会自动省略多的图片，不够严谨。
def show_in_one(images, show_size=(820, 820), blank_size=2, window_name="merge"):
    small_h, small_w = images[0].shape[:2]
    column = int(show_size[1] / (small_w + blank_size))
    row = int(show_size[0] / (small_h + blank_size))
    shape = [show_size[0], show_size[1]]
    for i in range(2, len(images[0].shape)):
        shape.append(images[0].shape[i])
    merge_img = np.zeros(tuple(shape), images[0].dtype)

    max_count = len(images)
    count = 0
    for i in range(row):
        if count >= max_count:
            break
        for j in range(column):
            if count < max_count:
                im = images[count]
                t_h_start = i * (small_h + blank_size)
                t_w_start = j * (small_w + blank_size)
                t_h_end = t_h_start + im.shape[0]
                t_w_end = t_w_start + im.shape[1]
                merge_img[t_h_start:t_h_end, t_w_start:t_w_end] = im
                count = count + 1
            else:
                break
    if count < max_count:
        print("图片总数为： %s" % (max_count - count))
    cv.namedWindow(window_name)
    cv.imshow(window_name, merge_img)

In [4]:
def showdiffpic(ref,res):
    reflist = ref.split(" ") #["./val_data/000.img",'num',x','y','w','h']
    reslist = res.split(" ")
    path = "./val_data/" + reflist[0]
    img = cv.imread(path)
    imgres = img.copy()
    #对ref画bbbox
    refnum = int(reflist[1])
    for i in range(refnum):
        #对检测到的每张脸算一下iou
        refrect = list(map(int,reflist[2 + 4 * i: (2 + 4 * i + 4)]))
        cv.rectangle(img,(refrect[0],refrect[1]),(refrect[0] + refrect[2],refrect[1] + refrect[3]),(255, 0, 0), 2)
    cv.namedWindow("the reference pic")
    cv.imshow("the reference pic",img)
    
    
    #对res画bbbox
    resnum = int(reslist[1])
    for i in range(resnum):
        #对检测到的每张脸算一下iou
        resrect = list(map(int,reslist[2 + 4 * i: (2 + 4 * i + 4)]))
        cv.rectangle(imgres,(resrect[0],resrect[1]),(resrect[0] + resrect[2],resrect[1] + resrect[3]),(255, 0, 0), 2)
    cv.namedWindow("the result_actual pic")
    cv.imshow("the result_actual pic",imgres)
    
    cv.waitKey(0)
    cv.destroyAllWindows()

showdiffpic("000001.jpg 1 40 74 100 100 ","000001.jpg 1 57 56 101 135")

In [5]:
def test_performance(refname,resname,areathreshold):
    reffilename = refname
    resfilename = resname
    reffile = open(reffilename)
    resfile = open(resfilename)
    TP = 0
    TN = 0
    FP = 0
    FN = 0
    #一行一行(一个例子一个例子)的读
    while(True):
        ref = reffile.readline().strip()
        res = resfile.readline().strip()
        if ref == '' or res == '':
            break
        if ref.split(" ")[0] != res.split(" ")[0]:
            print(ref.split(" ")[0])
            print(res.split(" ")[0])
            break
        #将文本分开
        reflist = ref.split(' ')[1:]
        reslist = res.split(' ')[1:]
        #先只写ref
        refdectect = int(reflist[0])
        resdectect = int(reslist[0])
        if resdectect == 0 or refdectect == 0:
            if refdectect == 0 and resdectect == 0:
                #FN
                FN += 1
                continue
            if refdectect == 0:
                #TN
                TN += 1
                continue
            else:
                #FP
                FP += 1
                showdiffpic(ref,res)
                continue
        else:
            #说明两个都检测到有物体
            if resdectect != refdectect:
                #说明两者检测到的物体数量不同
                FP += 1
                showdiffpic(ref,res)
                continue
            else:
                iouave = 0
                for i in range(resdectect):
                    #对检测到的每张脸算一下iou
                    refrect = map(int,reflist[1 + 4 * i: (1 + 4 * i + 4)])
                    resrect = map(int,reslist[1 + 4 * i: (1 + 4 * i + 4)])
                    #看两者的IOU有多大
                    area = iou(refrect,resrect)
                    iouave += area
                iouave /= resdectect
                if(iouave > areathreshold):
                    #TP
                    TP += 1
                else:
                    FP += 1
                    showdiffpic(ref,res)
                continue
    reffile.close()
    resfile.close()
    return {"TP":TP,"FP":FP,"TN":TN,"FN":FN}

In [6]:
os.path.split("a/b/c/d/e")

('a/b/c/d', 'e')

In [7]:
a = "a b c d\n"
print(a.split(" "))

['a', 'b', 'c', 'd\n']


test for the test_performance function
refname = "test_val_function/reffile.txt"
resname = "test_val_function/resfile.txt"
print(test_performance(refname,resname,0.8))

In [8]:
def iou(box1, box2):
    """Implement the intersection over union (IoU) between box1 and box2
    
    Arguments:
    box1 -- first box, list object with coordinates (x1, y1, x2, y2)
    box2 -- second box, list object with coordinates (x1, y1, x2, y2)
    """
    
    box1 = list(box1)
    box2 = list(box2)
    box1[2] += box1[0]
    box1[3] += box1[1]
    box2[2] += box2[0]
    box2[3] += box2[1]
    
    # Calculate the (y1, x1, y2, x2) coordinates of the intersection of box1 and box2. Calculate its Area.
    ### START CODE HERE ### (≈ 5 lines)
    xi1 = max(box1[0], box2[0])
    yi1 = max(box1[1], box2[1])
    xi2 = min(box1[2], box2[2])
    yi2 = min(box1[3], box2[3])
    if xi2 - xi1 > 0 and yi2 - yi1 > 0:
        inter_area =  (xi2-xi1) * (yi2-yi1)
    else:
         inter_area = 0
    ### END CODE HERE ###
    # Calculate the Union area by using Formula: Union(A,B) = A + B - Inter(A,B)
    ### START CODE HERE ### (≈ 3 lines)
    box1_area = (box1[2]-box1[0]) * (box1[3]-box1[1])
    box2_area = (box2[2]-box2[0]) * (box2[3]-box2[1])
    union_area = box1_area + box2_area - inter_area 
    ### END CODE HERE ###
    
    # compute the IoU
    ### START CODE HERE ### (≈ 1 line)
    iou = inter_area / union_area if inter_area > 0 else 0
    ### END CODE HERE ###
    
    return iou


In [9]:
#结果后处理
import pandas as pd
def postshow(resdic):
    TP = resdic['TP']
    FP = resdic['FP']
    TN = resdic["TN"]
    FN = resdic["FN"]
    result = pd.DataFrame(data = {"P":[TP,FP],"N":[TN,FN]},index = ["T","F"])
    print(result)
    print("the accuracy number is :",(TP+FN)/(TP+TN+FP+FN)*100,"%")
    print("查准率：",TP/(TP+TN)*100,"%")
    print("误检率: ",FP/(FP+FN)*100,"%")

In [14]:
import argparse
import glob
import numpy as np
import os
from pathlib import Path

if __name__ == '__main__':
#     parser = argparse.ArgumentParser(description='Demonstrate mouse interaction with images')
#     parser.add_argument("-i", "--input", help="Input directory.")
#     args = parser.parse_args()
#     path = args.input
#     if path is None:
#         # 修改为自己的图片存放的文件夹
#         test_dir = "./"
#         path = test_dir

    path = './val_data'
    debug_images = []
    i = 0
    #先清空输出
    f = open("./val_output/ref.txt","w")
    f.close()
    for infile in glob.glob(os.path.join(path,'*.*')):
        infile = Path(infile).as_posix()
        infile = os.path.join('./',infile)
        ext = os.path.splitext(infile)[1][1:]  # get the filename extenstion
        if ext == "png" or ext == "jpg" or ext == "bmp" or ext == "tiff" or ext == "pbm":
            #print(infile)
            img = cv.imread(infile)
            #img = cv.resize(img,(120,120))
            #对每张图片进行检测并且输出到ref.txt中
            detectface(img,infile)
            if img is None:
                continue
            else:
                debug_images.append(img)
    refname = "./val_output/ref.txt"
    resname = "./val_output/res.txt"
    resdic = test_performance(refname,resname,0.6)
    postshow(resdic)
    #show_in_one(debug_images)
   # cv.waitKey(0)
    cv.destroyAllWindows()

     P    N
T  314   30
F   98  195
the accuracy number is : 79.90580847723704 %
查准率： 91.27906976744185 %
误检率:  33.44709897610921 %
